### [How to use chat models to call tools](https://python.langchain.com/docs/how_to/tool_calling/)

In [1]:
import getpass
import os

if "LANGCHAIN_API_KEY" not in os.environ:
    os.environ["LANGCHAIN_TRACING_V2"] = "true"
    os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

In [2]:
if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass()

#### A. Python functions

In [1]:
def add(a: int, b: int) -> int:
    """Add two integers
    
    Args:
        a: First integer
        b: Second integer
    """
    return a + b

def multiply(a: int, b: int) -> int:
    """Multiply two integers
    
    Args:
        a: First integer
        b: Second integer
    """
    return a * b

#### B. Pydantic class

In [2]:
from pydantic import BaseModel, Field

#! Aqui los ... significan que es requerido
class add(BaseModel):
    """Add two integers"""

    a: int = Field(..., title="First integer")
    b: int = Field(..., title="Second integer")


class multiply(BaseModel):
    """Multiply two integers"""

    a: int = Field(..., title="First integer")
    b: int = Field(..., title="Second integer")

In [3]:
from typing_extensions import Annotated, TypedDict

#! Aqui los ... significan que no tiene valor por defecto
class add(TypedDict):
    """Add two integers."""

    # Annotations must have the type and can optionally include a default value and description (in that order).
    a: Annotated[int, ..., "First integer"]
    b: Annotated[int, ..., "Second integer"]


class multiply(TypedDict):
    """Multiply two integers."""

    a: Annotated[int, ..., "First integer"]
    b: Annotated[int, ..., "Second integer"]


tools = [add, multiply]

In [5]:
from langchain_openai import ChatOpenAI

tools = [add, multiply]

llm = ChatOpenAI(model='gpt-4o-mini')

llm_with_tools = llm.bind_tools(tools)

In [6]:
query = "What is 2 + 2?"
llm_with_tools.invoke(query)

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_AIM6oPWui6CAeBbD45kHBNjE', 'function': {'arguments': '{"a":2,"b":2}', 'name': 'add'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 87, 'total_tokens': 105, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0aa8d3e20b', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-8c7e11dc-5342-41d6-89b6-9b9594cb04d0-0', tool_calls=[{'name': 'add', 'args': {'a': 2, 'b': 2}, 'id': 'call_AIM6oPWui6CAeBbD45kHBNjE', 'type': 'tool_call'}], usage_metadata={'input_tokens': 87, 'output_tokens': 18, 'total_tokens': 105, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [8]:
# El modelo puede invocar a las herramientas
query = "What is 3 * 12? Also, what is 11 + 49?"

llm_with_tools.invoke(query).tool_calls

[{'name': 'multiply',
  'args': {'a': 3, 'b': 12},
  'id': 'call_CoyucKYCYN6MA31Otfjd9LoZ',
  'type': 'tool_call'},
 {'name': 'add',
  'args': {'a': 11, 'b': 49},
  'id': 'call_7QxPMBu7xO2YDqVPpY9nKzFm',
  'type': 'tool_call'}]

In [10]:
from langchain_core.output_parsers import PydanticToolsParser
from pydantic import BaseModel, Field

#! Se pueden convertir las herramientas a Pydantic
class add(BaseModel):
    """Add two integers."""

    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")


class multiply(BaseModel):
    """Multiply two integers."""

    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")


chain = llm_with_tools | PydanticToolsParser(tools=[add, multiply])
chain.invoke(query)

[multiply(a=3, b=12), add(a=11, b=49)]